# The Battle of the Neighborhoods – Week 1

## Introduction:
**_NextGen_** is a vegan restaurant that aims to recreate healthier verisons of common ‘comfort’, and ‘bar’ foods. **_NextGen’s_** menu is kept simple and affordable to attract younger crowds and those always on the go. Its mission is to make health foods easily accessible and to educate patrons on the importance of making better food choices daily. 


In [1]:
#Import libraries
import numpy as np #handles data in vactorized manner
import pandas as pd #handles data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json#handles JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim #converts an address into latitude and longitude values

import requests #handles requests
from pandas.io.json import json_normalize # transform JSON file into a pandas datafame

#Import matplotlib and plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Import k-means for clustering
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium #renders map

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



### Define Fourquare Credentials

In [97]:
# @hidden_cell
CLIENT_ID = 'Data Hidden'
CLIENT_SECRET = 'Data Hidden'
VERSION = '20180604'
LIMIT = 100

print('Credentials read in!')

### Geospatial Coordinates of Toronto

In [295]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


### Search for Universities

In [296]:
#Search for universities
search_query = 'University'
radius = 500

#Create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4N2JQWBURUPB3RAZRVHJB0GEWBEIY2XV1M52YICC1OEDQYZF&client_secret=CV1SIIOAW43MIMZ2BAWLMU4WOQ20GPE4T25HOHW024QEONS1&ll=43.6534817,-79.3839347&v=20180604&query=University&radius=500&limit=100'

In [297]:
#Send the GET Request 
results = requests.get(url).json()

#Assign relevant part of JSON to venues
venues = results['response']['venues']

#Tranform venues into a dataframe
uni_df = json_normalize(venues)
uni_df.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.postalCode,location.city,location.state,location.crossStreet,venuePage.id
0,4cd044c29d87224bf129543b,University Centre Gym,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1587928442,False,43.653571,-79.386979,"[{'label': 'display', 'lat': 43.653571, 'lng':...",245,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,NaN
1,4c6062b612e5c9286e131f4c,Dundas University Health Clinic,"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1587928442,False,43.654196,-79.388166,"[{'label': 'display', 'lat': 43.65419587934101...",349,CA,Canada,"[104-438 University Avenue, Toronto ON M4P 2K8...",104-438 University Avenue,M4P 2K8,Toronto,ON,NaN,NaN
2,4c45c6c6da2176b09543637a,University Centre,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1587928442,False,43.653907,-79.386764,"[{'label': 'display', 'lat': 43.65390719043468...",232,CA,Canada,"[393 University Ave. (at Armoury St.), Toronto...",393 University Ave.,M7A 2K1,Toronto,ON,at Armoury St.,NaN
3,4d5d275fff226a3110517e69,438 University Ave,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1587928442,False,43.654296,-79.388415,"[{'label': 'display', 'lat': 43.65429570830456...",372,CA,Canada,"[438 University Ave (at Dundas St W), Toronto ...",438 University Ave,M5G 2K8,Toronto,ON,at Dundas St W,NaN
4,4e6565fb1850e238d7048495,University Avenue Plaza,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",v-1587928442,False,43.652695,-79.387218,"[{'label': 'display', 'lat': 43.65269470686765...",278,CA,Canada,"[University Avenue (btwn Richmond & College), ...",University Avenue,NaN,Toronto,ON,btwn Richmond & College,NaN


#### The Foursquare API returned location data for The University of Toronto and Ryerson University.  

#### No data was returned for York University (ranked 3rd). Therefore, Ontario College of Art and Design University,(ranked 5th), will serve as top 3rd university, as Trent University (ranked 4th), is also not available.

### Clean the University Dataframe

In [307]:
#Keep columns that include venue name and any associated with location
uni_clean_col = ['name', 'categories'] + [col for col in uni_df.columns if col.startswith('location.')]+ ['id']
uni_clean_df = uni_df.loc[:,uni_clean_col]

#Retrieve venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#Filter category for each row
uni_clean_df['categories'] = uni_clean_df.apply(get_category_type, axis=1)

#Clean column names by removing the first term
uni_clean_df.columns = [column.split('.')[-1] for column in uni_clean_df.columns]
uni_clean_df.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,postalCode,city,state,crossStreet,id
0,University Centre Gym,Gym / Fitness Center,43.653571,-79.386979,"[{'label': 'display', 'lat': 43.653571, 'lng':...",245,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,4cd044c29d87224bf129543b
1,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,"[{'label': 'display', 'lat': 43.65419587934101...",349,CA,Canada,"[104-438 University Avenue, Toronto ON M4P 2K8...",104-438 University Avenue,M4P 2K8,Toronto,ON,NaN,4c6062b612e5c9286e131f4c
2,University Centre,Office,43.653907,-79.386764,"[{'label': 'display', 'lat': 43.65390719043468...",232,CA,Canada,"[393 University Ave. (at Armoury St.), Toronto...",393 University Ave.,M7A 2K1,Toronto,ON,at Armoury St.,4c45c6c6da2176b09543637a
3,438 University Ave,Office,43.654296,-79.388415,"[{'label': 'display', 'lat': 43.65429570830456...",372,CA,Canada,"[438 University Ave (at Dundas St W), Toronto ...",438 University Ave,M5G 2K8,Toronto,ON,at Dundas St W,4d5d275fff226a3110517e69
4,University Avenue Plaza,Plaza,43.652695,-79.387218,"[{'label': 'display', 'lat': 43.65269470686765...",278,CA,Canada,"[University Avenue (btwn Richmond & College), ...",University Avenue,NaN,Toronto,ON,btwn Richmond & College,4e6565fb1850e238d7048495


In [308]:
#Delete unnecessary columns
uni_clean_df2= uni_clean_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance','formattedAddress', 'id','labeledLatLngs'], axis=1)
uni_clean_df2.head()

,name,categories,lat,lng,address,postalCode,state
0,University Centre Gym,Gym / Fitness Center,43.653571,-79.386979,NaN,NaN,NaN
1,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,104-438 University Avenue,M4P 2K8,ON
2,University Centre,Office,43.653907,-79.386764,393 University Ave.,M7A 2K1,ON
3,438 University Ave,Office,43.654296,-79.388415,438 University Ave,M5G 2K8,ON
4,University Avenue Plaza,Plaza,43.652695,-79.387218,University Avenue,NaN,ON


In [309]:
#Delete rows with NaN values
uni_clean_df3 = uni_clean_df2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) 
uni_clean_df3

,name,categories,lat,lng,address,postalCode,state
1,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,104-438 University Avenue,M4P 2K8,ON
2,University Centre,Office,43.653907,-79.386764,393 University Ave.,M7A 2K1,ON
3,438 University Ave,Office,43.654296,-79.388415,438 University Ave,M5G 2K8,ON
6,393 University Ave,Office,43.653752,-79.388150,393 University Ave,M5G 1E6,ON
9,480 University Ave,Office,43.655099,-79.388648,480 University Ave,M5G 1V2,ON
12,Ontario College of Art and Design University (...,University,43.652803,-79.391074,100 McCaul St,M5T 1W1,ON
14,425 University Avenue,Office,43.653769,-79.383949,425 University Avenue,M5G 1T6,ON
28,University Health Network Spiritual Care Depar...,Spiritual Center,43.658209,-79.384962,700 Bay St,M5G 0A2,ON
30,University Settlement Recreation Centre,College Gym,43.652139,-79.391477,23 Grange Rd.,M5T 1C3,ON
31,Ryerson University,University,43.657935,-79.380490,350 Victoria St.,M5B 2K3,ON


#### The location data does not display direct location information for The University of Toronto. However, the addresses that include ('University Ave') are within The University of Toronto campus boundaries. 

#### For purposes of further analysis, **Dundas & University** in the 'Intersection' category will be used as the University of Toronto's main address. This replacement locaiton identifier is approx. 3 minutes driving distance from the offical campus address. 

### Clean and Isolate Top 3 Universities Dataframe

In [310]:
#Isolate the top 3 university locations by selecting where category is equal to Intersection or University
array= ['Intersection', 'University']
topuni_df= uni_clean_df3.loc[uni_clean_df3['categories'].isin(array)]

#Replace Ontario College of Art and Design University with acronym 'OCAD' University
topuni_df.loc[12, 'name'] = 'OCAD University'

#Replace 'Dundas & University' name with 'University of Toronto'
topuni_df.loc[39,'name'] = 'University of Toronto'
topuni_df

,name,categories,lat,lng,address,postalCode,state
12,OCAD University,University,43.652803,-79.391074,100 McCaul St,M5T 1W1,ON
31,Ryerson University,University,43.657935,-79.380490,350 Victoria St.,M5B 2K3,ON
39,University of Toronto,Intersection,43.654620,-79.389150,University Ave,M5G 1V1,ON


### Search for Gyms

In [290]:
#Search for Gyms
search_query = ['Fitness','Training']
radius = 500

#Create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

"https://api.foursquare.com/v2/venues/search?client_id=4N2JQWBURUPB3RAZRVHJB0GEWBEIY2XV1M52YICC1OEDQYZF&client_secret=CV1SIIOAW43MIMZ2BAWLMU4WOQ20GPE4T25HOHW024QEONS1&ll=43.6534817,-79.3839347&v=20180604&query=['Fitness', 'Training']&radius=500&limit=100"

In [291]:
#Send the GET Request 
gym_results = requests.get(url).json()

#Assign relevant part of JSON to venues
venues = gym_results['response']['venues']

#Tranform venues into a dataframe
gym_df = json_normalize(venues)
gym_df.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4ff8e46fe4b03705cefc7075,Fitness Centre,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1587928358,False,525 Bay St.,in Marriott Downtown,43.654690,-79.381739,"[{'label': 'display', 'lat': 43.65469045595554...",222,M5G 2L2,CA,Toronto,ON,Canada,"[525 Bay St. (in Marriott Downtown), Toronto O...",NaN,NaN
1,4b5243a7f964a5209d7327e3,Fitness Centre,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1587928358,False,Sheraton Centre,123 Queen St W,43.650985,-79.384002,"[{'label': 'display', 'lat': 43.65098505166592...",277,M5H 2M9,CA,Toronto,ON,Canada,"[Sheraton Centre (123 Queen St W), Toronto ON ...",NaN,NaN
2,53451cff498e555a4e340937,doubletree fitness center (Doubletree Fitness ...,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1587928358,False,108 Chestnut Street,Dundas St W,43.654603,-79.386204,"[{'label': 'display', 'lat': 43.65460313972433...",221,M5G 1R3,CA,Toronto,ON,Canada,"[108 Chestnut Street (Dundas St W), Toronto ON...",NaN,NaN
3,50885719498ea7b5aab3a74c,GoodLife Fitness Toronto Bell Trinity Centre,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1587928358,False,483 Bay St,NaN,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",130,M5G 2C9,CA,Toronto,ON,Canada,"[483 Bay St, Toronto ON M5G 2C9, Canada]",NaN,NaN
4,5ba17f68911fc4002c36a07d,Busy Bee Fitness Experts,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1587928358,False,319 Merton Street,Mount Pleasant,43.653944,-79.383370,"[{'label': 'display', 'lat': 43.65394410274548...",68,M4S 1A5,CA,Toronto,ON,Canada,"[319 Merton Street (Mount Pleasant), Toronto O...",NaN,NaN


### Clean the Gym Dataframe

In [292]:
#Keep columns that include venue name and any associated with location
gym_clean_col = ['name', 'categories'] + [col for col in gym_df.columns if col.startswith('location.')]+ ['id']
gym_clean_df = gym_df.loc[:,gym_clean_col]

#Retrieve venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#Filter category for each row
gym_clean_df['categories'] = gym_clean_df.apply(get_category_type, axis=1)

#Clean column names by removing the first term
gym_clean_df.columns = [column.split('.')[-1] for column in gym_clean_df.columns]
gym_clean_df.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Fitness Centre,Gym,525 Bay St.,in Marriott Downtown,43.654690,-79.381739,"[{'label': 'display', 'lat': 43.65469045595554...",222,M5G 2L2,CA,Toronto,ON,Canada,"[525 Bay St. (in Marriott Downtown), Toronto O...",NaN,4ff8e46fe4b03705cefc7075
1,Fitness Centre,Gym,Sheraton Centre,123 Queen St W,43.650985,-79.384002,"[{'label': 'display', 'lat': 43.65098505166592...",277,M5H 2M9,CA,Toronto,ON,Canada,"[Sheraton Centre (123 Queen St W), Toronto ON ...",NaN,4b5243a7f964a5209d7327e3
2,doubletree fitness center (Doubletree Fitness ...,Gym / Fitness Center,108 Chestnut Street,Dundas St W,43.654603,-79.386204,"[{'label': 'display', 'lat': 43.65460313972433...",221,M5G 1R3,CA,Toronto,ON,Canada,"[108 Chestnut Street (Dundas St W), Toronto ON...",NaN,53451cff498e555a4e340937
3,GoodLife Fitness Toronto Bell Trinity Centre,Gym,483 Bay St,NaN,43.653436,-79.382314,"[{'label': 'display', 'lat': 43.653436, 'lng':...",130,M5G 2C9,CA,Toronto,ON,Canada,"[483 Bay St, Toronto ON M5G 2C9, Canada]",NaN,50885719498ea7b5aab3a74c
4,Busy Bee Fitness Experts,Gym / Fitness Center,319 Merton Street,Mount Pleasant,43.653944,-79.383370,"[{'label': 'display', 'lat': 43.65394410274548...",68,M4S 1A5,CA,Toronto,ON,Canada,"[319 Merton Street (Mount Pleasant), Toronto O...",NaN,5ba17f68911fc4002c36a07d


In [293]:
#Delete unnecessary columns
gym_clean_df2= gym_clean_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance','formattedAddress', 'id','labeledLatLngs', 'neighborhood'], axis=1)
gym_clean_df2.head()

,name,categories,address,lat,lng,postalCode,state
0,Fitness Centre,Gym,525 Bay St.,43.654690,-79.381739,M5G 2L2,ON
1,Fitness Centre,Gym,Sheraton Centre,43.650985,-79.384002,M5H 2M9,ON
2,doubletree fitness center (Doubletree Fitness ...,Gym / Fitness Center,108 Chestnut Street,43.654603,-79.386204,M5G 1R3,ON
3,GoodLife Fitness Toronto Bell Trinity Centre,Gym,483 Bay St,43.653436,-79.382314,M5G 2C9,ON
4,Busy Bee Fitness Experts,Gym / Fitness Center,319 Merton Street,43.653944,-79.383370,M4S 1A5,ON


In [294]:
#Delete rows with NaN values
gym_clean_df3 = gym_clean_df2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) 
gym_clean_df3

,name,categories,address,lat,lng,postalCode,state
0,Fitness Centre,Gym,525 Bay St.,43.654690,-79.381739,M5G 2L2,ON
1,Fitness Centre,Gym,Sheraton Centre,43.650985,-79.384002,M5H 2M9,ON
2,doubletree fitness center (Doubletree Fitness ...,Gym / Fitness Center,108 Chestnut Street,43.654603,-79.386204,M5G 1R3,ON
3,GoodLife Fitness Toronto Bell Trinity Centre,Gym,483 Bay St,43.653436,-79.382314,M5G 2C9,ON
4,Busy Bee Fitness Experts,Gym / Fitness Center,319 Merton Street,43.653944,-79.383370,M4S 1A5,ON
5,Google Toronto Fitness Centre,Gym / Fitness Center,"111 Richmond St W, 11F",43.650319,-79.383782,M5H 2G4,ON
8,GoodLife Fitness Toronto McCaul and Queen,Gym,21 McCaul St,43.651148,-79.389683,M5T 1V7,ON
10,GoodLife Fitness Toronto 137 Yonge Street,Gym,137 Yonge St,43.651242,-79.378068,M5C 1W6,ON
11,Flex - Personal Training Delivered to You,Gym,250 Yonge St,43.654196,-79.381714,M5B 2M6,ON
12,GoodLife Fitness Toronto Plaza,Gym,100 Yonge St,43.649742,-79.378537,M5C 2W1,ON


In [271]:
#Isolate the top 3 university locations by selecting where category is equal to Intersection or University
array= ['Intersection', 'University']
gym_df= gym_clean_df3.loc[gym_clean_df3['categories'].isin(array)]
gym_df

,name,categories,lat,lng,address,state,postalCode


### Search for Bars

In [197]:
#Search for Bars
search_query = 'Bar'
radius = 500

#Create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4N2JQWBURUPB3RAZRVHJB0GEWBEIY2XV1M52YICC1OEDQYZF&client_secret=CV1SIIOAW43MIMZ2BAWLMU4WOQ20GPE4T25HOHW024QEONS1&ll=43.6534817,-79.3839347&v=20180604&query=Bar&radius=500&limit=100'

In [198]:
#Send the GET Request 
bar_results = requests.get(url).json()

#Assign relevant part of JSON to venues
venues = bar_results['response']['venues']

#Tranform venues into a dataframe
bar_df = json_normalize(venues)
bar_df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.neighborhood,venuePage.id
0,57dd99cb498ee67580d16390,Bar Verde,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",v-1587923552,False,260 Yonge Street,43.654837,-79.380742,"[{'label': 'display', 'lat': 43.654837, 'lng':...",298,M5B 2L9,CA,Toronto,ON,Canada,"[260 Yonge Street, Toronto ON M5B 2L9, Canada]",NaN,NaN,NaN
1,4af8da92f964a5204a1022e3,Blue Stone Grill & Bar,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1587923552,False,372 Bay St.,43.651187,-79.381139,"[{'label': 'display', 'lat': 43.65118713330148...",340,M5H 1M7,CA,Toronto,ON,Canada,"[372 Bay St. (at Richmond St.), Toronto ON M5H...",at Richmond St.,NaN,NaN
2,4b3db5e4f964a520709625e3,Quinn's Steakhouse & Bar,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",v-1587923552,False,96 Richmond Street West,43.651197,-79.382976,"[{'label': 'display', 'lat': 43.65119745750837...",265,M5H 2A3,CA,Toronto,ON,Canada,"[96 Richmond Street West (at Bay), Toronto ON ...",at Bay,Financial District,90490783
3,50ca02be245f2d4aa8c2ab5b,Barristers Bar,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1587923552,False,145 Richmond Street West,43.649797,-79.385807,"[{'label': 'display', 'lat': 43.64979669604817...",437,M5H 2L2,CA,Toronto,ON,Canada,"[145 Richmond Street West (Hilton Toronto), To...",Hilton Toronto,NaN,NaN
4,4b6a0069f964a520f3c02be3,Pantages Lounge & Bar,"[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",v-1587923552,False,200 Victoria St.,43.654493,-79.379000,"[{'label': 'display', 'lat': 43.65449343126956...",413,M5B 1W8,CA,Toronto,ON,Canada,"[200 Victoria St. (at Shuter St.), Toronto ON ...",at Shuter St.,NaN,NaN


### Clean the Bar Dataframe

In [108]:
#Keep columns that include venue name and any associated with location
clean_col = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_df = dataframe.loc[:,clean_col]

#Retrieve venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#Filter category for each row
clean_df['categories'] = clean_df.apply(get_category_type, axis=1)

#Clean column names by removing the first term
clean_df.columns = [column.split('.')[-1] for column in clean_df.columns]
clean_df.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,postalCode,city,state,crossStreet,id
0,University Centre Gym,Gym / Fitness Center,43.653571,-79.386979,"[{'label': 'display', 'lat': 43.653571, 'lng':...",245,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,4cd044c29d87224bf129543b
1,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,"[{'label': 'display', 'lat': 43.65419587934101...",349,CA,Canada,"[104-438 University Avenue, Toronto ON M4P 2K8...",104-438 University Avenue,M4P 2K8,Toronto,ON,NaN,4c6062b612e5c9286e131f4c
2,438 University Ave,Office,43.654296,-79.388415,"[{'label': 'display', 'lat': 43.65429570830456...",372,CA,Canada,"[438 University Ave (at Dundas St W), Toronto ...",438 University Ave,M5G 2K8,Toronto,ON,at Dundas St W,4d5d275fff226a3110517e69
3,University Centre,Office,43.653907,-79.386764,"[{'label': 'display', 'lat': 43.65390719043468...",232,CA,Canada,"[393 University Ave. (at Armoury St.), Toronto...",393 University Ave.,M7A 2K1,Toronto,ON,at Armoury St.,4c45c6c6da2176b09543637a
4,University Avenue Plaza,Plaza,43.652695,-79.387218,"[{'label': 'display', 'lat': 43.65269470686765...",278,CA,Canada,"[University Avenue (btwn Richmond & College), ...",University Avenue,NaN,Toronto,ON,btwn Richmond & College,4e6565fb1850e238d7048495


In [191]:
#Delete unnecessary columns
clean_df2= clean_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance','formattedAddress', 'id','labeledLatLngs'], axis=1)
clean_df2.head()

,name,categories,lat,lng,address,postalCode,state
0,University Centre Gym,Gym / Fitness Center,43.653571,-79.386979,NaN,NaN,NaN
1,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,104-438 University Avenue,M4P 2K8,ON
2,438 University Ave,Office,43.654296,-79.388415,438 University Ave,M5G 2K8,ON
3,University Centre,Office,43.653907,-79.386764,393 University Ave.,M7A 2K1,ON
4,University Avenue Plaza,Plaza,43.652695,-79.387218,University Avenue,NaN,ON


In [192]:
#Delete rows with NaN values
clean_df3 = clean_df2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) 
clean_df3

,name,categories,lat,lng,address,postalCode,state
1,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,104-438 University Avenue,M4P 2K8,ON
2,438 University Ave,Office,43.654296,-79.388415,438 University Ave,M5G 2K8,ON
3,University Centre,Office,43.653907,-79.386764,393 University Ave.,M7A 2K1,ON
6,393 University Ave,Office,43.653752,-79.388150,393 University Ave,M5G 1E6,ON
9,480 University Ave,Office,43.655099,-79.388648,480 University Ave,M5G 1V2,ON
12,Ontario College of Art and Design University (...,University,43.652803,-79.391074,100 McCaul St,M5T 1W1,ON
14,425 University Avenue,Office,43.653769,-79.383949,425 University Avenue,M5G 1T6,ON
27,University Health Network Spiritual Care Depar...,Spiritual Center,43.658209,-79.384962,700 Bay St,M5G 0A2,ON
30,University Settlement Recreation Centre,College Gym,43.652139,-79.391477,23 Grange Rd.,M5T 1C3,ON
31,"Regus - Ontario, Toronto - University & Dundas",Business Service,43.654400,-79.387900,"439 University Avenue, 5th Floor",M5G 1Y8,ON


In [183]:
#Isolate the top 3 university locations by selecting where category is equal to Intersection or University
array= ['Intersection', 'University']
topuni_df= clean_df3.loc[clean_df3['categories'].isin(array)]

#Replace Ontario College of Art and Design University with acronym 'OCAD' University
topuni_df.loc[12, 'name'] = 'OCAD University'

#Replace 'Dundas & University' name with 'University of Toronto'
topuni_df.loc[49,'name'] = 'University of Toronto'
topuni_df

,name,categories,lat,lng,address,postalCode,state
12,OCAD University,University,43.652803,-79.391074,100 McCaul St,M5T 1W1,ON
40,Ryerson University,University,43.657935,-79.380490,350 Victoria St.,M5B 2K3,ON
49,University of Toronto,Intersection,43.654620,-79.389150,University Ave,M5G 1V1,ON


### Search for Shopping Malls

In [236]:
#Search for shopping malls
search_query = 'Shopping'
radius = 1000

#Create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4N2JQWBURUPB3RAZRVHJB0GEWBEIY2XV1M52YICC1OEDQYZF&client_secret=CV1SIIOAW43MIMZ2BAWLMU4WOQ20GPE4T25HOHW024QEONS1&ll=43.6534817,-79.3839347&v=20180604&query=Shopping&radius=1000&limit=100'

In [237]:
#Send the GET Request 
mall_results = requests.get(url).json()

#Assign relevant part of JSON to venues
venues = mall_results['response']['venues']

#Tranform venues into a dataframe
mall_df = json_normalize(venues)
mall_df

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,5840cafe06f1a34af80cc609,Saks Fifth Avenue Club - Personal Shopping,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",v-1587925623,False,176 Yonge Street,Queen Street West,43.651810,-79.379192,"[{'label': 'display', 'lat': 43.65181, 'lng': ...",424,M5C 2L7,CA,Toronto,ON,Canada,"[176 Yonge Street (Queen Street West), Toronto...",NaN
1,4ad77a12f964a520260b21e3,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1587925623,False,220 Yonge St,btwn Queen & Dundas,43.654540,-79.380677,NaN,287,M5B 2H1,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...","Downtown Toronto, Toronto, ON"
2,4b9e86dff964a520e6eb36e3,TD Centre Shopping Concourse,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",v-1587925623,False,66 Wellington St W,btw York & Bay St,43.647184,-79.380932,"[{'label': 'display', 'lat': 43.64718424611984...",741,M5K 1A1,CA,Toronto,ON,Canada,"[66 Wellington St W (btw York & Bay St), Toron...",NaN


### Clena the Shopping Mall Dataframe

In [108]:
#Keep columns that include venue name and any associated with location
clean_col = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_df = dataframe.loc[:,clean_col]

#Retrieve venue category 
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#Filter category for each row
clean_df['categories'] = clean_df.apply(get_category_type, axis=1)

#Clean column names by removing the first term
clean_df.columns = [column.split('.')[-1] for column in clean_df.columns]
clean_df.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,postalCode,city,state,crossStreet,id
0,University Centre Gym,Gym / Fitness Center,43.653571,-79.386979,"[{'label': 'display', 'lat': 43.653571, 'lng':...",245,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,4cd044c29d87224bf129543b
1,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,"[{'label': 'display', 'lat': 43.65419587934101...",349,CA,Canada,"[104-438 University Avenue, Toronto ON M4P 2K8...",104-438 University Avenue,M4P 2K8,Toronto,ON,NaN,4c6062b612e5c9286e131f4c
2,438 University Ave,Office,43.654296,-79.388415,"[{'label': 'display', 'lat': 43.65429570830456...",372,CA,Canada,"[438 University Ave (at Dundas St W), Toronto ...",438 University Ave,M5G 2K8,Toronto,ON,at Dundas St W,4d5d275fff226a3110517e69
3,University Centre,Office,43.653907,-79.386764,"[{'label': 'display', 'lat': 43.65390719043468...",232,CA,Canada,"[393 University Ave. (at Armoury St.), Toronto...",393 University Ave.,M7A 2K1,Toronto,ON,at Armoury St.,4c45c6c6da2176b09543637a
4,University Avenue Plaza,Plaza,43.652695,-79.387218,"[{'label': 'display', 'lat': 43.65269470686765...",278,CA,Canada,"[University Avenue (btwn Richmond & College), ...",University Avenue,NaN,Toronto,ON,btwn Richmond & College,4e6565fb1850e238d7048495


In [191]:
#Delete unnecessary columns
clean_df2= clean_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance','formattedAddress', 'id','labeledLatLngs'], axis=1)
clean_df2.head()

,name,categories,lat,lng,address,postalCode,state
0,University Centre Gym,Gym / Fitness Center,43.653571,-79.386979,NaN,NaN,NaN
1,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,104-438 University Avenue,M4P 2K8,ON
2,438 University Ave,Office,43.654296,-79.388415,438 University Ave,M5G 2K8,ON
3,University Centre,Office,43.653907,-79.386764,393 University Ave.,M7A 2K1,ON
4,University Avenue Plaza,Plaza,43.652695,-79.387218,University Avenue,NaN,ON


In [192]:
#Delete rows with NaN values
clean_df3 = clean_df2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False) 
clean_df3

,name,categories,lat,lng,address,postalCode,state
1,Dundas University Health Clinic,Medical Center,43.654196,-79.388166,104-438 University Avenue,M4P 2K8,ON
2,438 University Ave,Office,43.654296,-79.388415,438 University Ave,M5G 2K8,ON
3,University Centre,Office,43.653907,-79.386764,393 University Ave.,M7A 2K1,ON
6,393 University Ave,Office,43.653752,-79.388150,393 University Ave,M5G 1E6,ON
9,480 University Ave,Office,43.655099,-79.388648,480 University Ave,M5G 1V2,ON
12,Ontario College of Art and Design University (...,University,43.652803,-79.391074,100 McCaul St,M5T 1W1,ON
14,425 University Avenue,Office,43.653769,-79.383949,425 University Avenue,M5G 1T6,ON
27,University Health Network Spiritual Care Depar...,Spiritual Center,43.658209,-79.384962,700 Bay St,M5G 0A2,ON
30,University Settlement Recreation Centre,College Gym,43.652139,-79.391477,23 Grange Rd.,M5T 1C3,ON
31,"Regus - Ontario, Toronto - University & Dundas",Business Service,43.654400,-79.387900,"439 University Avenue, 5th Floor",M5G 1Y8,ON


In [183]:
#Isolate the top 3 university locations by selecting where category is equal to Intersection or University
array= ['Intersection', 'University']
topuni_df= clean_df3.loc[clean_df3['categories'].isin(array)]

#Replace Ontario College of Art and Design University with acronym 'OCAD' University
topuni_df.loc[12, 'name'] = 'OCAD University'

#Replace 'Dundas & University' name with 'University of Toronto'
topuni_df.loc[49,'name'] = 'University of Toronto'
topuni_df

,name,categories,lat,lng,address,postalCode,state
12,OCAD University,University,43.652803,-79.391074,100 McCaul St,M5T 1W1,ON
40,Ryerson University,University,43.657935,-79.380490,350 Victoria St.,M5B 2K3,ON
49,University of Toronto,Intersection,43.654620,-79.389150,University Ave,M5G 1V1,ON


## To Do:

### Universities identified
* Clean all dfs
* Merge all dfs
* Cluster map